# Nbody simulation

Get Acceleration:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')

"""
Create Your Own N-body Simulation (With Python)
Philip Mocz (2020) Princeton Univeristy, @PMocz
Simulate orbits of stars interacting due to gravity
Code calculates pairwise forces according to Newton's Law of Gravity
"""

def getAcc( pos, mass, G, softening ):
	"""
    Calculate the acceleration on each particle due to Newton's Law 
	pos  is an N x 3 matrix of positions
	mass is an N x 1 vector of masses
	G is Newton's Gravitational constant
	softening is the softening length
	a is N x 3 matrix of accelerations
	"""
	# positions r = [x,y,z] for all particles
	x = pos[:,0:1]
	y = pos[:,1:2]
	z = pos[:,2:3]

	# matrix that stores all pairwise particle separations: r_j - r_i
	dx = x.T - x
	dy = y.T - y
	dz = z.T - z

	# matrix that stores 1/r^3 for all particle pairwise particle separations 
	inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
	inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)

	ax = G * (dx * inv_r3) @ mass
	ay = G * (dy * inv_r3) @ mass
	az = G * (dz * inv_r3) @ mass
	
	# pack together the acceleration components
	a = np.hstack((ax,ay,az))

	return a

## Energy:

In [ ]:
def getEnergy( pos, vel, mass, G ):
	"""
	Get kinetic energy (KE) and potential energy (PE) of simulation
	pos is N x 3 matrix of positions
	vel is N x 3 matrix of velocities
	mass is an N x 1 vector of masses
	G is Newton's Gravitational constant
	KE is the kinetic energy of the system
	PE is the potential energy of the system
	"""
	# Kinetic Energy:
	KE = 0.5 * np.sum(np.sum( mass * vel**2 ))


	# Potential Energy:

	# positions r = [x,y,z] for all particles
	x = pos[:,0:1]
	y = pos[:,1:2]
	z = pos[:,2:3]

	# matrix that stores all pairwise particle separations: r_j - r_i
	dx = x.T - x
	dy = y.T - y
	dz = z.T - z

	# matrix that stores 1/r for all particle pairwise particle separations 
	inv_r = np.sqrt(dx**2 + dy**2 + dz**2)
	inv_r[inv_r>0] = 1.0/inv_r[inv_r>0]

	# sum over upper triangle, to count each interaction only once
	PE = G * np.sum(np.sum(np.triu(-(mass*mass.T)*inv_r,1)))
	
	return KE, PE;


Main simulation:

In [ ]:

def main():
	""" N-body simulation """
	
	# Simulation parameters
	N         = 100    # Number of particles
	t         = 0      # current time of the simulation
	tEnd      = 10.0   # time at which simulation ends
	dt        = 0.01   # timestep
	softening = 0.1    # softening length
	G         = 1.0    # Newton's Gravitational Constant
	plotRealTime = True # switch on for plotting as the simulation goes along
	
	# Generate Initial Conditions
	np.random.seed(17)            # set the random number generator seed
	
	mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
	pos  = np.random.randn(N,3)   # randomly selected positions and velocities
	vel  = np.random.randn(N,3)
	
	# Convert to Center-of-Mass frame
	vel -= np.mean(mass * vel,0) / np.mean(mass)
	
	# calculate initial gravitational accelerations
	acc = getAcc( pos, mass, G, softening )
	
	# calculate initial energy of system
	KE, PE  = getEnergy( pos, vel, mass, G )
	
	# number of timesteps
	Nt = int(np.ceil(tEnd/dt))
	
	# save energies, particle orbits for plotting trails
	pos_save = np.zeros((N,3,Nt+1))
	pos_save[:,:,0] = pos


	# prep figure
	fig = plt.figure(figsize=(4,5), dpi=80)
	grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
	ax = plt.subplot(grid[0:2,0])
	
	# Simulation Main Loop
	for i in range(Nt):
		# (1/2) kick
		vel += acc * dt/2.0
		
		# drift
		pos += vel * dt
		
		# update accelerations
		acc = getAcc( pos, mass, G, softening )
		
		# (1/2) kick
		vel += acc * dt/2.0
		
		# update time
		t += dt
		
		# get energy of system
		KE, PE  = getEnergy( pos, vel, mass, G )
		
		
		# save energies, positions for plotting trail
		pos_save[:,:,i+1] = pos

		
		# plot in real time
		if plotRealTime or (i == Nt-1):
			plt.sca(ax)
			plt.cla()
			xx = pos_save[:,0,max(i-50,0):i+1]
			yy = pos_save[:,1,max(i-50,0):i+1]
			plt.scatter(xx,yy,s=1,color=[.7,.7,1])
			plt.scatter(pos[:,0],pos[:,1],s=10,color='blue')
			ax.set(xlim=(-2, 2), ylim=(-2, 2))
			ax.set_aspect('equal', 'box')
			ax.set_xticks([-2,-1,0,1,2])
			ax.set_yticks([-2,-1,0,1,2])
			plt.pause(0.001)
			clear_output(wait=True)
	
	
	# Save figure
	plt.savefig('nbody.png',dpi=240)
	plt.show()

	plt.close()
	
	return 0

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
import matplotlib.animation as animation 

In [ ]:
def main_pyplot_animation():
	""" N-body simulation """
	
	# Simulation parameters
	N         = 100    # Number of particles
	t         = 0      # current time of the simulation
	tEnd      = 10.0   # time at which simulation ends
	dt        = 0.01   # timestep
	softening = 0.1    # softening length
	G         = 1.0    # Newton's Gravitational Constant
	plotRealTime = True # switch on for plotting as the simulation goes along
	
	# Generate Initial Conditions
	np.random.seed(17)            # set the random number generator seed
	
	mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
	pos  = np.random.randn(N,3)   # randomly selected positions and velocities
	vel  = np.random.randn(N,3)
	
	# Convert to Center-of-Mass frame
	vel -= np.mean(mass * vel,0) / np.mean(mass)
	
	# calculate initial gravitational accelerations
	acc = getAcc( pos, mass, G, softening )
		
	# number of timesteps
	Nt = int(np.ceil(tEnd/dt))
	
	# save energies, particle orbits for plotting trails
	pos_save = np.zeros((N,3,Nt+1))
	pos_save[:,:,0] = pos


	# prep figure
	fig = plt.figure(figsize=(4,5), dpi=80)
	grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
	ax = plt.subplot(grid[0:10,0])
	line, = ax.plot([], [], lw=3)

	
	# Simulation Main Loop
	for i in range(Nt):
		# (1/2) kick
		vel += acc * dt/2.0
		
		# drift
		pos += vel * dt
		
		# update accelerations
		acc = getAcc( pos, mass, G, softening )
		
		# (1/2) kick
		vel += acc * dt/2.0
		
		# update time
		t += dt		
		
		# save energies, positions for plotting trail
		pos_save[:,:,i+1] = pos

		def init():
			line.set_data([], [])
			return line,
		
		# plot in real time
		def animate(frame):
			#plt.sca(ax)
			plt.cla()
			xx = pos_save[:,0,max(i-50,0):i+1]
			yy = pos_save[:,1,max(i-50,0):i+1]
			plt.scatter(xx,yy,s=1,color=[.7,.7,1])
			plt.scatter(pos[:,0],pos[:,1],s=10,color='blue')
			plt.pause(0.01)
			line.set_data(xx, yy)
			return line,
	
		anim = animation.FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)
		f = r"c://Users/filip/Desktop/animation.gif" 
		writergif = animation.PillowWriter(fps=30) 
		anim.save(f, writer=writergif)

	return 0

In [ ]:
main_pyplot_animation()

In [ ]:
""" N-body simulation """
def getAcc( pos, mass, G, softening ):
	"""
    Calculate the acceleration on each particle due to Newton's Law 
	pos  is an N x 3 matrix of positions
	mass is an N x 1 vector of masses
	G is Newton's Gravitational constant
	softening is the softening length
	a is N x 3 matrix of accelerations
	"""
	# positions r = [x,y,z] for all particles
	x = pos[:,0:1]
	y = pos[:,1:2]
	z = pos[:,2:3]

	# matrix that stores all pairwise particle separations: r_j - r_i
	dx = x.T - x
	dy = y.T - y
	dz = z.T - z

	# matrix that stores 1/r^3 for all particle pairwise particle separations 
	inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)
	inv_r3[inv_r3>0] = inv_r3[inv_r3>0]**(-1.5)

	ax = G * (dx * inv_r3) @ mass
	ay = G * (dy * inv_r3) @ mass
	az = G * (dz * inv_r3) @ mass
	
	# pack together the acceleration components
	a = np.hstack((ax,ay,az))

	return a

# Simulation parameters
N         = 100    # Number of particles
t         = 0      # current time of the simulation
tEnd      = 10.0   # time at which simulation ends
dt        = 0.01   # timestep
softening = 0.1    # softening length
G         = 1.0    # Newton's Gravitational Constant
plotRealTime = True # switch on for plotting as the simulation goes along
	
# Generate Initial Conditions
np.random.seed(17)            # set the random number generator seed
	
mass = 20.0*np.ones((N,1))/N  # total mass of particles is 20
pos  = np.random.randn(N,3)   # randomly selected positions and velocities
vel  = np.random.randn(N,3)
	
# Convert to Center-of-Mass frame
vel -= np.mean(mass * vel,0) / np.mean(mass)
	
# calculate initial gravitational accelerations
acc = getAcc( pos, mass, G, softening )
		
# number of timesteps
Nt = int(np.ceil(tEnd/dt))
	
# save energies, particle orbits for plotting trails
pos_save = np.zeros((N,3,Nt+1))
pos_save[:,:,0] = pos


# prep figure
fig = plt.figure(figsize=(4,5), dpi=80)
grid = plt.GridSpec(3, 1, wspace=0.0, hspace=0.3)
ax = plt.subplot(grid[0:2,0])
line, = ax.plot([], [], lw=3)

	
# Simulation Main Loop
for i in range(Nt):
	# (1/2) kick
	vel += acc * dt/2.0
		
	# drift
	pos += vel * dt
		
	# update accelerations
	acc = getAcc( pos, mass, G, softening )
		
	# (1/2) kick
	vel += acc * dt/2.0
		
	# update time
	t += dt		
		
	# save energies, positions for plotting trail
	pos_save[:,:,i+1] = pos

	def init():
		line.set_data([], [])
		return line,
	
	# plot in real time
	def animate(frame):
		plt.sca(ax)
		plt.cla()
		xx = pos_save[:,0,max(i-50,0):i+1]
		yy = pos_save[:,1,max(i-50,0):i+1]
		#plt.scatter(xx,yy,s=1,color=[.7,.7,1])
		#plt.scatter(pos[:,0],pos[:,1],s=10,color='blue')
		line.set_data(xx, yy, s=1,color=[.7,.7,1])
		plt.show()
		plt.pause(0.001)
		return line,
	anim = animation.FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)

'''
	f = r"c://Users/filip/Desktop/animation.gif" 
	writergif = animation.PillowWriter(fps=30) 
	anim.save(f, writer=writergif)
	
'''